## Introducción

Guia completa: https://www.pluralsight.com/guides/classification-keras

Keras es una API de redes neuronales de alto nivel, escrita en Python, y puede ejecutarse sobre TensorFlow, CNTK o Theano. Una de las ventajas es que permite una primera aproximación al deep learning. 

Las RN consumen muchos mas recursos de cómputo y por eso solo se usan en situaciones que lo ameritan. Ademas, el algoritmo que se genera en las capas intermedias es prácticamente inaccesible y por lo tanto no se tiene un control sobre el mismo como el que si tendríamos con otros algoritmos.

Planteo este ejercicio opcional sobre RN porque es un algoritmo muy popular y seria interesante analizar el desempeño sobre nuestro dataset.

In [10]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.metrics import Recall

# Obtenemos los datos

df_etiquetados = []
df_no_etiquetados = []
filedir = "data_preprocessed"

from os import listdir

for file in listdir(filedir):
    if file[-4:] == ".csv":
        if "noEtiquetado" in file: df_no_etiquetados.append(pd.read_csv(f"{filedir}/{file}"))
        else: df_etiquetados.append(pd.read_csv(f"{filedir}/{file}"))
    else: print(file, " ---> no es csv")

for df in df_etiquetados: assert len(df.columns) == 11
for df in df_no_etiquetados: assert len(df.columns) == 10
    
df = pd.concat(df_etiquetados)

### Filtro de outliers

In [11]:
# Aplico un filtro a cada columna y reasigno con el df original.
def outlier_filter(df, c, low, top):
    return df[(df[c] > low) & (df[c] < top)]


# Elimino outliers para cada columna con metodo de cuantiles
for col in df.columns:
    low = df[col].quantile(0.01)
    top = df[col].quantile(0.99)
    dff = outlier_filter(df, col, low, top)
    
# Elimino nulos
dff.dropna(inplace=True)

df = dff
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5849 entries, 0 to 2393
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Patient  5849 non-null   float64
 1   State    5849 non-null   float64
 2   PE       5849 non-null   float64
 3   SC       5849 non-null   float64
 4   GNE      5849 non-null   float64
 5   SE       5849 non-null   float64
 6   LZC      5849 non-null   float64
 7   STE      5849 non-null   float64
 8   Mean     5849 non-null   float64
 9   Std      5849 non-null   float64
 10  Skew     5849 non-null   float64
dtypes: float64(11)
memory usage: 548.3 KB


<ipython-input-11-80ecba153e75>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff.dropna(inplace=True)


In [12]:
# Separamos features de etiqueta
X = df.drop(labels=['State','Patient'], axis=1)
y = df['State']

In [45]:
# Train and validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0, shuffle=False)

## Instanciacion del modelo

Vamos a usar el constructor Sequential porque nuestra RN consiste en una pila de capas lineales. 

Las segunda linea de codigo representa la capa de entrada que especifica la *funcion de activacion* y la cantidad de dimensiones de entrada, que en nuestro caso corresponde a los 9 features. Repetimos este proceso para dos capas ocultas siguientes, sin el parametro de numero de inputs. Usamos como funcion de activacion a la unidad de rectificacion lineal ReLU (pueden elegir otra).

La ultima capa es la de salida y tiene dos nodos porque tenemos dos etiquetas: 0 y 1. Usamos softmax como funcion de activacion para la capa de salida par aque la suma de todos los valores predecidos de todos los nodos en la capa de salida suma 1.

La funcion compile() configura el proceso de aprendizaje y estan seteados algunos parametros. Usamos categorical_crossentropy como funcion de perdida, 'adam' el algoritmo de optimizacion, y 'recall' nuestra metrica de evaluacion.

In [46]:
model = Sequential()
model.add(Dense(500, activation='relu', input_dim=9))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

Instanciamos el modelo con 20 epochs

In [47]:
# build the model
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
128/128 [==============================] - 0s 977us/step - loss: 0.7010 - accuracy: 0.5161
Epoch 2/20
128/128 [==============================] - 0s 891us/step - loss: 0.6931 - accuracy: 0.4685
Epoch 3/20
128/128 [==============================] - 0s 1ms/step - loss: 0.6931 - accuracy: 0.4902
Epoch 4/20
128/128 [==============================] - 0s 906us/step - loss: 0.6931 - accuracy: 0.5017
Epoch 5/20
128/128 [==============================] - 0s 875us/step - loss: 0.6931 - accuracy: 0.4983
Epoch 6/20
128/128 [==============================] - 0s 863us/step - loss: 0.6931 - accuracy: 0.5017
Epoch 7/20
128/128 [==============================] - 0s 871us/step - loss: 0.6931 - accuracy: 0.4922
Epoch 8/20
128/128 [==============================] - 0s 871us/step - loss: 0.6931 - accuracy: 0.5024
Epoch 9/20
128/128 [==============================] - 0s 879us/step - loss: 0.6932 - accuracy: 0.4951
Epoch 10/20
128/128 [==============================] - 0s 879us/step - loss: 0.6932 

## Evaluación

The first line of code predicts on the train data, while the second line evaluates the model, and the third line prints the accuracy and error on the training data.

The same is repeated in the fourth, fifth and sixth lines of code which is performed on the test data.

In [48]:
pred_train= model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))   

Accuracy on training data: 0.8971666097640991% 
 Error on training data: 0.10283339023590088
Accuracy on test data: 0.9475783705711365% 
 Error on test data: 0.052421629428863525


Tenemos una accuracy del 90% para train y un 94% para test (sospechosamente alto). Fijense que ocurre cuando cambiamos el Shuffle=True en el train_test_split. Que esta pasando?

### Consigna

- Aplicar este modelo sobre los datos dentro de sus criterios de outliers / curacion y balanceo de datos.
- investigar sobre las mejores configuraciones del compiler para nuestros datos: optimizer, loss y metrics. 
- Evaluar las predicciones.